# Introduction
In this captone project, I will analyze districts in the city of Los Angeles: I will group similar districts in terms of their venue distribution. By doing so, a future entreprenuer should be able to tell most favorable locations in launching his or her business. For example, when a businessman desires to open up a cafe, it would be more beneficial if he does so in the urban cities than rural areas. 

To contrive the data analysis, I would 1)extract data showing coordinates based on different zipodes of LA from "opendatasoft.com" 2)based on the coordinates, retrieve the necessary data using FourSquare API 3)retrieve the district's neiborhood information(e.g. the number of restaurants and parks) 4) lastly, conduct K-Means clustering using venue frequency as input data. 


In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim 
import folium